## **TRI DE DECHETS**

---

R.Q. Ce notebook a été lancé sur google colab. L'importation des données a été faite via google drive.

### **I. DATA PRE PROCESSING**

### **1. Import the dataset**

In [ ]:
# import data
from google.colab import drive
import glob
import os

# transformation & preprocessing
import cv2
import random
from PIL import Image
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as implt
from tensorflow.keras.utils import to_categorical

# train test
from sklearn.model_selection import train_test_split

# increasing data
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# training model & regularisation
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from keras.layers import BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

# Model for transfer learning
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2

In [ ]:
# Import the dataset
drive.mount('/content/drive')
image_base_path = '/content/drive/My Drive/dataset-master'

### **2. Transformation**

In [ ]:
# Class names
class_names = ['crumpled_paper', 'disposable_paper_cups', 'egg_packaging', 'foil', 'glass_bottle', 'plastic_bottle', 'receipt']

# Mapping des noms de classe à des entiers
class_mapping = {class_name: i for i, class_name in enumerate(class_names)}

trash_images = []
labels = []

def load_and_preprocess_image(image_path):
    image_path = str(image_path)
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if image is None:
        print(f"Error loading image: {image_path}")
        return None
    image = cv2.resize(image, (128, 128))       # Taille fixe (par exemple, 128x128)
    image = image.astype('float32') / 255.0     # Convertit l'image en tableau numpy et normaliser les valeurs de pixels
    return image

image_base_path = '/content/drive/My Drive/dataset-master'


# Collecter les images et les étiquettes
for label, trash_folder in enumerate(class_names):
    trash_folder_path = os.path.join(image_base_path, trash_folder)
    if os.path.isdir(trash_folder_path):
        for image_file in os.listdir(trash_folder_path):
            if image_file.endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(trash_folder_path, image_file)
                image = load_and_preprocess_image(image_path)
                if image is not None:
                    trash_images.append(image)
                    labels.append(label)
                else:
                    print(f"Failed to load image: {image_path}")  # Debugging statement
    else:
        print(f"Directory does not exist: {trash_folder_path}")   # Debugging statement


# Convertit les listes en tableaux numpy
trash_images = np.array(trash_images)
labels = np.array(labels).astype(np.int64)

print(f"Total images loaded: {len(trash_images)}")  # Debugging statement

# Check if any images were loaded
if len(trash_images) == 0:
    raise ValueError("No images were loaded. Please check your image path and data structure.")

In [ ]:
# Train-test split (80% train + val, 20% test)
train_val_images, test_images, train_val_labels, test_labels = train_test_split(trash_images, labels, test_size=0.2, random_state=42)

# Train-validation split (80% train, 20% val de l'ensemble train + val, donc 64% train, 16% (0.8*0.2) val du total)
train_images, val_images, train_labels, val_labels = train_test_split(train_val_images, train_val_labels, test_size=0.2, random_state=42)

# One-hot encoding des labels
train_labels = to_categorical(train_labels, num_classes=len(class_names))   #### train_val_labels ?
val_labels = to_categorical(val_labels, num_classes=len(class_names))
test_labels = to_categorical(test_labels, num_classes=len(class_names))

print(f"Total images: {len(trash_images)}")
print(f"Train images: {len(train_images)}")
print(f"Validation images: {len(val_images)}")
print(f"Test images: {len(test_images)}")

### **3. Visualisation**

In [ ]:
# class_names
def show_images(images, labels, class_names, num_images=10):
    num_images = min(num_images, len(images))

    plt.figure(figsize=(30, 20))

    for i in range(num_images):
        plt.subplot(1, num_images, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i])
        label_index = np.argmax(labels[i])
        plt.xlabel(class_names[label_index])

    plt.show()

# Affiche les 10 premières images d'entraînement et leurs étiquettes correspondantes
show_images(train_val_images, train_labels, class_names, num_images=10)

### **II. TRAINING AND EVALUATION**

### **1. CNN modèle**

In [ ]:
def build_cnn_model():
  """
  Cette fonction nous permet de créer un modèle CNN avec des couches personnalisées
  """

  cnn_model = Sequential([
      Conv2D(32, (3, 3), padding = 'same', activation='relu', input_shape=(128, 128, 3), kernel_regularizer=l2(0.05)),       #, kernel_regularizer=l2(0.01)
      BatchNormalization(),
      # Dropout(0.3),
      MaxPooling2D((2, 2)),

      Conv2D(64, (3, 3), padding = 'same', activation='relu', kernel_regularizer=l2(0.05)),                                  #, kernel_regularizer=l2(0.01)
      BatchNormalization(),
      # Dropout(0.3),
      MaxPooling2D((2, 2)),

      Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.05)),
      BatchNormalization(),
      MaxPooling2D((2, 2)),

      Flatten(),
      Dense(128, activation='relu', kernel_regularizer=l2(0.05)),

      Dropout(0.5),   # 0.2
      Dense(len(class_names), activation='softmax')
  ])

  return cnn_model

cnn_model = build_cnn_model()

# Initialise le modèle en lui passant une donnée
cnn_model.predict(train_images[[0]])

In [ ]:
# Compile le modèle
cnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# Ajout de earlystopping, et reduce_lr afin de limiter l'overfitting
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=32)   #, restore_best_weights=True
mc = ModelCheckpoint('cnn_model_best.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1) 

# Entraîne le modèle
history = cnn_model.fit(train_images, train_labels, 
                        epochs=40, 
                        validation_data=(val_images, val_labels), 
                        batch_size=32, 
                        callbacks=[es, mc, reduce_lr])   # , reduce_lr

> Les scores sont aux alentours de :  loss: 0.4917 - accuracy: 0.9709 - val_loss: 1.2728 - val_accuracy: 0.7297.

In [ ]:
# Évalue le modèle sur les données de test
test_loss, test_accuracy = cnn_model.evaluate(test_images, test_labels)
print("Précision sur les données de test :", test_accuracy)

On va par la suite opter pour le transfer learning soit utiliser des modèles déjà pré-entraînés.

### **2. VGG19**

In [ ]:
vgg19_model = keras.applications.VGG19(
    input_shape=(128,128,3),
    include_top=False,
    weights="imagenet"
)

In [ ]:
# vgg19_model.summary()

# Gel des couches de base du modèle
vgg19_model.trainable = False

# Ajout des couches de classification
x = vgg19_model.output
x = GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)      ##
x = Dense(128, activation='relu', kernel_regularizer=l2(0.05))(x)   ##
x = Dropout(0.3)(x)
predictions = Dense(len(class_names), activation='softmax')(x)

# Crée le modèle final
transf_vgg19_model = tf.keras.Model(vgg19_model.input, predictions)

transf_vgg19_model.compile(optimizer='adam',              # tf.keras.optimizers.Adam(learning_rate=1e-3)
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])

# Ajout de earlystopping, et reduce_lr afin de limiter l'overfitting
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=32)   #, restore_best_weights=True
mc = ModelCheckpoint('cnn_model_best.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1) 

history = transf_vgg19_model.fit(train_images, train_labels, 
                                 validation_data=(val_images, val_labels), 
                                 batch_size=32, 
                                 epochs=40, 
                                 callbacks=[es, mc, reduce_lr])

**FINE TUNE THE MODEL**

In [ ]:
# Dégel des dernières couches du modèle de base
vgg19_model.trainable = True     # transf_vgg19_model ou vgg19_model

# Nombre de couches à dégeler
fine_tune_at = 100

# Gèle toutes les couches avant 'fine_tune_at'
for layer in vgg19_model.layers[:fine_tune_at]:
    layer.trainable = False

# Compilation
transf_vgg19_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),     # taux d'apprentissage plus faible
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_fine = transf_vgg19_model.fit(train_images, train_labels,
                                  validation_data=(val_images, val_labels),
                                  batch_size=32,
                                  epochs=40,
                                  callbacks=[es, mc])      # , reduce_lr

### **3. EfficientNetb2**

In [ ]:
base_model = tf.keras.applications.EfficientNetB2(
    input_shape=(128,128,3),
    include_top=False,        # on va définir des couches personnalisées ci après
    weights="imagenet"
)

# base_model.summary()

In [ ]:
# Gel des couches de base du modèle
base_model.trainable = False

# Ajout de couches personnalisées de classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)                # , kernel_regularizer=tf.keras.regularizers.l2(0.01)
# x = Dropout(0.3)(x)
predictions = Dense(len(class_names), activation='softmax')(x)

# Crée le modèle final
transf_model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
transf_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Add earlystopping
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=32)    # , restore_best_weights=True
mc = ModelCheckpoint('efficientnetb2_model_best.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1)

# Training
history = transf_model.fit(train_images, train_labels, 
                           validation_data=(val_images, val_labels), 
                           batch_size=32, 
                           epochs=40, 
                           callbacks=[es, mc])      # , reduce_lr

# Evaluation
test_loss, test_acc = transf_model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

> Ce modèle est beaucoup moins performant que les autres.

### **III. MAKE PREDICTIONS**

### **1. CNN MODEL**

In [ ]:
# Charger le modèle enregistré si l'entrainement a déjà ete fait
cnn_model_best = tf.keras.models.load_model('/content/cnn_model_best.h5')

In [ ]:
predictions = cnn_model.predict(test_images)

# Predicted label pour la 1ère image
predictions[0]

# Classe qui a le taux de confiance le plus élevé
prediction = np.argmax(predictions[0])
print(class_names[prediction])

# pour vérifier si la prédiction est correcte
print("Label of this image is:",class_names[np.argmax(test_labels[0])])
plt.imshow(test_images[0])

In [ ]:
# Fonctions pour afficher les prédictions avec la probabilité de confiance
def plot_image_prediction(i, predictions_array, class_names, true_label, img):
  
  predictions_array, true_label, img = predictions_array[i], np.argmax(true_label[i]), img[i]

  plt.imshow(img)

  predicted_label = np.argmax(predictions_array)

  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)



def plot_value_prediction(i, predictions_array, class_names, true_label):
  
  predictions_array, true_label = predictions_array[i], np.argmax(true_label[i])
  thisplot = plt.bar(range(len(class_names)), predictions_array, color="#777777")

  plt.ylim([0, 1])
  
  predicted_label = np.argmax(predictions_array)
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

  plt.xticks(range(len(class_names)), 
             class_names, 
             rotation=45, 
             ha='right')


In [ ]:
# Pour voir les prédictions du modèle
#@title Changer le slider pour observer les prédictions du modèle CNN classique! { run: "auto" }

image_index = 96 #@param {type:"slider", min:0, max:100, step:1}

plt.subplot(1,2,1)
plot_image_prediction(image_index, predictions, class_names, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_prediction(image_index, predictions, class_names, test_labels)

In [ ]:
# Plots the first X test images, their predicted label, and the true label
# Color correct predictions in blue, incorrect predictions in red
num_rows = 5
num_cols = 4
num_images = num_rows*num_cols

plt.figure(figsize=(2*2*num_cols, 2*num_rows))

for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image_prediction(i, predictions, class_names,test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_prediction(i, predictions, class_names,test_labels)
  plt.tight_layout(pad=2.0)  